<a href="https://colab.research.google.com/github/Kinrider/TechChallenge/blob/main/03%20-%20Scripts/ETL_exp_prod_venti_della_valle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas

In [ ]:
import pandas as pd
import random as rd
import numpy as np

# Define as opções para exibir floats sem notação científica

pd.set_option('display.float_format', '{:.0f}'.format)

# Tratamento de dados de Exportação

In [ ]:
# Para ler arquivos do Git, sempre específicar no final : '?raw=true'

dados = 'https://github.com/Kinrider/TechChallenge/blob/cad34f20bb46229c8cc8b78d6a21143aa68d6726/Fontes/ExpVinho.csv?raw=true'

In [ ]:
vinhosexp = pd.read_csv(dados, sep=';' , encoding= 'utf8')
vinhosexp
x = vinhosexp.copy()

In [ ]:
x.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,3660,25467,6261,32605,2698,6741,7630,45367,4806,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,345,1065,0,0,0,0,4068,4761,0,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
array_dollar=['Id' , 'País']
array_kilos=['Id' , 'País']

for column in x.columns:
  if '.1' in column:
   dollar = column.replace('.1' , '')+'_'+'dollar'
   x.rename({column:dollar},inplace=True, axis=1)
   array_dollar.append(dollar)
  else:
    kilos = column +'_'+ 'kilos'
    x.rename({column:kilos},inplace=True, axis=1)
    array_kilos.append(kilos)

In [ ]:
x.rename({'Id_kilos' :'Id'},inplace=True, axis=1)
x.rename({'País_kilos':'País'},inplace=True, axis=1)

In [ ]:
array_kilos.remove('Id_kilos')
array_kilos.remove('País_kilos')

In [ ]:
x[array_kilos].head()

,Id,País,1970_kilos,1971_kilos,1972_kilos,1973_kilos,1974_kilos,1975_kilos,1976_kilos,1977_kilos,...,2014_kilos,2015_kilos,2016_kilos,2017_kilos,2018_kilos,2019_kilos,2020_kilos,2021_kilos,2022_kilos,2023_kilos
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,0,26,4,0,0,117
2,3,"Alemanha, República Democrática",0,0,4168,12000,0,0,0,0,...,213348,10680,14012,15467,10794,3660,6261,2698,7630,4806
3,4,Angola,0,0,0,0,0,0,0,0,...,12182,1908,7359,10170,477,345,0,0,4068,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x[array_dollar].head()

,Id,País,1970_dollar,1971_dollar,1972_dollar,1973_dollar,1974_dollar,1975_dollar,1976_dollar,1977_dollar,...,2014_dollar,2015_dollar,2016_dollar,2017_dollar,2018_dollar,2019_dollar,2020_dollar,2021_dollar,2022_dollar,2023_dollar
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,46,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,0,95,21,0,0,698
2,3,"Alemanha, República Democrática",0,0,2630,8250,0,0,0,0,...,761653,44780,68109,87702,45382,25467,32605,6741,45367,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,23124,17089,35390,61680,709,1065,0,0,4761,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = x

In [ ]:
# Reshape do dataframe

# 1º transpor kilos para linha

df1 = df.melt(
    id_vars = ['Id', 'País'],
    value_vars = df.filter(like = '_kilos').columns,
    var_name = 'ano',
    value_name = 'kilos'
    )

# 2º transpor dólar para linha

df2 = df.melt(
    id_vars = ['Id', 'País'],
    value_vars = df.filter(like = '_dollar').columns,
    var_name = 'anos',
    value_name = 'dollar'
    )

# 3º sincronizar os índices

df1 = df1.set_index(['Id','País', df1.groupby(['Id','País']).cumcount()])
df2 = df2.set_index(['Id','País', df1.groupby(['Id','País']).cumcount()])

# 4º Juntar data_frames

df3 = (
    pd.concat([df1, df2], axis=1)
  .sort_index(level=2)
  .reset_index(level=2, drop=True)
  .reset_index()
  )

df3.head()

# referências:
# https://stackoverflow.com/questions/51519101/simultaneously-melt-multiple-columns-in-python-pandas
# https://pandas.pydata.org/docs/reference/api/pandas.melt.html
# https://medium.com/horadecodar/data-science-tips-02-como-usar-loc-e-iloc-no-pandas-fab58e214d87

,Id,País,ano,kilos,anos,dollar
0,1,Afeganistão,1970_kilos,0,1970_dollar,0
1,2,África do Sul,1970_kilos,0,1970_dollar,0
2,3,"Alemanha, República Democrática",1970_kilos,0,1970_dollar,0
3,4,Angola,1970_kilos,0,1970_dollar,0
4,5,Anguilla,1970_kilos,0,1970_dollar,0


In [ ]:
# Dropa coluna de ano duplicada

df3.drop(['anos'], axis=1, inplace = True)

In [ ]:
# tratando o campo de ano

df3['ano'] = df3['ano'].str.replace('_kilos','') + '-01-01'

df3.head()

,Id,País,ano,kilos,dollar
0,1,Afeganistão,1970-01-01,0,0
1,2,África do Sul,1970-01-01,0,0
2,3,"Alemanha, República Democrática",1970-01-01,0,0
3,4,Angola,1970-01-01,0,0
4,5,Anguilla,1970-01-01,0,0


In [ ]:
# tranformando string em data

df3['ano'] = pd.to_datetime(df3['ano'],format='%Y-%m-%d')

df3.head()

,Id,País,ano,kilos,dollar
0,1,Afeganistão,1970-01-01,0,0
1,2,África do Sul,1970-01-01,0,0
2,3,"Alemanha, República Democrática",1970-01-01,0,0
3,4,Angola,1970-01-01,0,0
4,5,Anguilla,1970-01-01,0,0


In [ ]:
# fitrando dados de 2008 pra frente:

df3.query('ano >= "2008-01-01"' ,inplace = True)

In [ ]:
df3.head()

,Id,País,ano,kilos,dollar
5206,1,Afeganistão,2008-01-01,0,0
5207,2,África do Sul,2008-01-01,0,0
5208,3,"Alemanha, República Democrática",2008-01-01,265742,429970
5209,4,Angola,2008-01-01,25721,71083
5210,5,Anguilla,2008-01-01,0,0


In [ ]:
# Verificando nomenclatura dos países

df3['País'].unique()

array(['Afeganistão', 'África do Sul', 'Alemanha, República Democrática',
       'Angola', 'Anguilla', 'Antígua e Barbuda', 'Antilhas Holandesas',
       'Arábia Saudita', 'Argentina', 'Aruba', 'Austrália', 'Áustria',
       'Bahamas', 'Bangladesh', 'Barbados', 'Barein', 'Bélgica', 'Belice',
       'Benin', 'Bermudas', 'Bolívia', 'Bósnia-Herzegovina', 'Brasil',
       'Bulgária', 'Cabo Verde', 'Camarões', 'Canadá', 'Catar',
       'Cayman, Ilhas', 'Chile', 'China', 'Chipre', 'Cingapura',
       'Cocos (Keeling), Ilhas', 'Colômbia', 'Comores', 'Congo',
       'Coreia, Republica Sul', 'Costa do Marfim', 'Costa Rica',
       'Croácia', 'Cuba', 'Curaçao', 'Dinamarca', 'Dominica',
       'El Salvador', 'Emirados Arabes Unidos', 'Equador',
       'Eslovaca, Republica', 'Espanha', 'Estados Unidos', 'Estônia',
       'Filipinas', 'Finlândia', 'França', 'Gana', 'Gibraltar', 'Granada',
       'Grécia', 'Guatemala', 'Guiana', 'Guiana Francesa', 'Guine Bissau',
       'Guine Equatorial', 'Haiti', 

In [ ]:
# Solicitado ao GPT que gerasse a lista de de_para, considerando os países listados acima

lista_paises = [
    ('Afeganistão', 'Afghanistan', 'AFG'),
    ('África do Sul', 'South Africa', 'ZAF'),
    ('Alemanha, República Democrática', 'Germany', 'DEU'),
    ('Angola', 'Angola', 'AGO'),
    ('Anguilla', 'Anguilla', 'AIA'),
    ('Antígua e Barbuda', 'Antigua and Barbuda', 'ATG'),
    ('Antilhas Holandesas', 'Netherlands Antilles', 'ANT'),
    ('Arábia Saudita', 'Saudi Arabia', 'SAU'),
    ('Argentina', 'Argentina', 'ARG'),
    ('Aruba', 'Aruba', 'ABW'),
    ('Austrália', 'Australia', 'AUS'),
    ('Áustria', 'Austria', 'AUT'),
    ('Bahamas', 'Bahamas', 'BHS'),
    ('Bangladesh', 'Bangladesh', 'BGD'),
    ('Barbados', 'Barbados', 'BRB'),
    ('Barein', 'Bahrain', 'BHR'),
    ('Bélgica', 'Belgium', 'BEL'),
    ('Belice', 'Belize', 'BLZ'),
    ('Benin', 'Benin', 'BEN'),
    ('Bermudas', 'Bermuda', 'BMU'),
    ('Bolívia', 'Bolivia', 'BOL'),
    ('Bósnia-Herzegovina', 'Bosnia and Herzegovina', 'BIH'),
    ('Brasil', 'Brazil', 'BRA'),
    ('Bulgária', 'Bulgaria', 'BGR'),
    ('Cabo Verde', 'Cabo Verde', 'CPV'),
    ('Camarões', 'Cameroon', 'CMR'),
    ('Canadá', 'Canada', 'CAN'),
    ('Catar', 'Qatar', 'QAT'),
    ('Cayman, Ilhas', 'Cayman Islands', 'CYM'),
    ('Chile', 'Chile', 'CHL'),
    ('China', 'China, mainland', 'CHN'),
    ('Chipre', 'Cyprus', 'CYP'),
    ('Cingapura', 'Singapore', 'SGP'),
    ('Cocos (Keeling), Ilhas', 'Cocos (Keeling) Islands', 'CCK'),
    ('Colômbia', 'Colombia', 'COL'),
    ('Comores', 'Comoros', 'COM'),
    ('Congo', 'Congo', 'COG'),
    ('Coreia, Republica Sul', 'Korea (the Republic of)', 'KOR'),
    ('Costa do Marfim', "Côte d'Ivoire", 'CIV'),
    ('Costa Rica', 'Costa Rica', 'CRI'),
    ('Croácia', 'Croatia', 'HRV'),
    ('Cuba', 'Cuba', 'CUB'),
    ('Curaçao', 'Curacao', 'CUW'),
    ('Dinamarca', 'Denmark', 'DNK'),
    ('Dominica', 'Dominica', 'DMA'),
    ('El Salvador', 'El Salvador', 'SLV'),
    ('Emirados Arabes Unidos', 'United Arab Emirates', 'ARE'),
    ('Equador', 'Ecuador', 'ECU'),
    ('Eslovaca, Republica', 'Slovakia', 'SVK'),
    ('Espanha', 'Spain', 'ESP'),
    ('Estados Unidos', 'United States of America', 'USA'),
    ('Estônia', 'Estonia', 'EST'),
    ('Filipinas', 'Philippines', 'PHL'),
    ('Finlândia', 'Finland', 'FIN'),
    ('França', 'France', 'FRA'),
    ('Gana', 'Ghana', 'GHA'),
    ('Gibraltar', 'Gibraltar', 'GIB'),
    ('Granada', 'Grenada', 'GRD'),
    ('Grécia', 'Greece', 'GRC'),
    ('Guatemala', 'Guatemala', 'GTM'),
    ('Guiana', 'Guyana', 'GUY'),
    ('Guiana Francesa', 'French Guiana', 'GUF'),
    ('Guine Bissau', 'Guinea-Bissau', 'GNB'),
    ('Guine Equatorial', 'Equatorial Guinea', 'GNQ'),
    ('Haiti', 'Haiti', 'HTI'),
    ('Honduras', 'Honduras', 'HND'),
    ('Hong Kong', 'China, Hong Kong SAR', 'HKG'),
    ('Hungria', 'Hungary', 'HUN'),
    ('Ilha de Man', 'Isle of Man', 'IMN'),
    ('Ilhas Virgens', 'Virgin Islands', 'VIR'),
    ('India', 'India', 'IND'),
    ('Indonésia', 'Indonesia', 'IDN'),
    ('Irã', 'Iran', 'IRN'),
    ('Iraque', 'Iraq', 'IRQ'),
    ('Irlanda', 'Ireland', 'IRL'),
    ('Itália', 'Italy', 'ITA'),
    ('Jamaica', 'Jamaica', 'JAM'),
    ('Japão', 'Japan', 'JPN'),
    ('Jordânia', 'Jordan', 'JOR'),
    ('Letônia', 'Latvia', 'LVA'),
    ('Líbano', 'Lebanon', 'LBN'),
    ('Libéria', 'Liberia', 'LBR'),
    ('Luxemburgo', 'Luxemburg', 'LUX'),
    ('Macau', 'China, Macao SAR', 'MAC'),
    ('Malásia', 'Malaysia', 'MYS'),
    ('Malavi', 'Malawi', 'MWI'),
    ('Malta', 'Malta', 'MLT'),
    ('Marshall, Ilhas', 'Marshall Islands', 'MHL'),
    ('Martinica', 'Martinique', 'MTQ'),
    ('Mauritânia', 'Mauritania', 'MRT'),
    ('México', 'Mexico', 'MEX'),
    ('Moçambique', 'Mozambique', 'MOZ'),
    ('Montenegro', 'Montenegro', 'MNE'),
    ('Namíbia', 'Namibia', 'NAM'),
    ('Nicarágua', 'Nicaragua', 'NIC'),
    ('Nigéria', 'Nigeria', 'NGA'),
    ('Noruega', 'Norway', 'NOR'),
    ('Nova Caledônia', 'New Caledonia', 'NCL'),
    ('Nova Zelândia', 'New Zealand', 'NZL'),
    ('Omã', 'Oman', 'OMN'),
    ('Países Baixos', 'Netherlands', 'NLD'),
    ('Palau', 'Palau', 'PLW'),
    ('Panamá', 'Panama', 'PAN'),
    ('Paraguai', 'Paraguay', 'PRY'),
    ('Peru', 'Peru', 'PER'),
    ('Pitcairn', 'Pitcairn Islands', 'PCN'),
    ('Polônia', 'Poland', 'POL'),
    ('Porto Rico', 'Puerto Rico', 'PRI'),
    ('Portugal', 'Portugal', 'PRT'),
    ('Quênia', 'Kenya', 'KEN'),
    ('Reino Unido', 'United Kingdom', 'GBR'),
    ('República Dominicana', 'Dominican Republic', 'DOM'),
    ('Rússia', 'Russia', 'RUS'),
    ('São Cristóvão e Névis', 'Saint Kitts and Nevis', 'KNA'),
    ('São Tomé e Príncipe', 'Sao Tome and Principe', 'STP'),
    ('São Vicente e Granadinas', 'Saint Vincent and the Grenadines', 'VCT'),
    ('Senegal', 'Senegal', 'SEN'),
    ('Serra Leoa', 'Sierra Leone', 'SLE'),
    ('Singapura', 'Singapore', 'SGP'),
    ('Suazilândia', 'Eswatini', 'SWZ'),
    ('Suécia', 'Sweden', 'SWE'),
    ('Suíça', 'Switzerland', 'CHE'),
    ('Suriname', 'Suriname', 'SUR'),
    ('Tailândia', 'Thailand', 'THA'),
    ('Taiwan (Formosa)', 'China, Taiwan Province of', 'TWN'),
    ('Tanzânia', 'Tanzania, the United Republic of', 'TZA'),
    ('Tcheca, República', 'Czech Republic', 'CZE'),
    ('Togo', 'Togo', 'TGO'),
    ('Toquelau', 'Tokelau', 'TKL'),
    ('Trinidade Tobago', 'Trinidad and Tobago', 'TTO'),
    ('Tunísia', 'Tunisia', 'TUN'),
    ('Turquia', 'Republic of Türkiye', 'TUR'),
    ('Tuvalu', 'Tuvalu', 'TUV'),
    ('Uruguai', 'Uruguay', 'URY'),
    ('Vanuatu', 'Vanuatu', 'VUT'),
    ('Venezuela', 'Venezuela', 'VEN'),
    ('Vietnã', 'Vietnam', 'VNM')
]

# Tranformando a tupla de países em um dataframe:

de_para_pais = pd.DataFrame(lista_paises, columns = ['pais_pt','pais_in','cod_pais'])

In [ ]:
# join com de_para de paises

df_exp = df3.merge(de_para_pais, how = 'left', left_on=['País'],right_on = ['pais_pt'])

In [ ]:
# Dropa colunas não utilizadas:

df_exp.drop(['País','pais_in','Id'], axis=1, inplace = True)

In [ ]:
df_exp.head()

,ano,kilos,dollar,pais_pt,cod_pais
0,2008-01-01,0,0,Afeganistão,AFG
1,2008-01-01,0,0,África do Sul,ZAF
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU
3,2008-01-01,25721,71083,Angola,AGO
4,2008-01-01,0,0,Anguilla,AIA


#Estimando Exportacao Venti della valle

In [ ]:
df_exp.head()

,ano,kilos,dollar,pais_pt,cod_pais
0,2008-01-01,0,0,Afeganistão,AFG
1,2008-01-01,0,0,África do Sul,ZAF
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU
3,2008-01-01,25721,71083,Angola,AGO
4,2008-01-01,0,0,Anguilla,AIA


In [ ]:
# Define uma função que vai aplicar sobre o total um percentual entre 18% a 24% do volume total para a nossa vinícola

# Define uma função que vai aplicar sobre o total um percentual entre 18% a 24% do volume total para a nossa vinícola

def mult(x,y):
  f = (rd.randint(18, 24)/100)
  return (f * x , f * y)


In [ ]:
# Reinicia o índice para futura iteração

df_exp.reset_index(drop=True)

,ano,kilos,dollar,pais_pt,cod_pais
0,2008-01-01,0,0,Afeganistão,AFG
1,2008-01-01,0,0,África do Sul,ZAF
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU
3,2008-01-01,25721,71083,Angola,AGO
4,2008-01-01,0,0,Anguilla,AIA
...,...,...,...,...,...
2187,2023-01-01,0,0,Tuvalu,TUV
2188,2023-01-01,326093,454271,Uruguai,URY
2189,2023-01-01,0,0,Vanuatu,VUT
2190,2023-01-01,141030,220512,Venezuela,VEN


In [ ]:
#Itera sobre o index, de forma a aplicar para cada index na coluna criada chamada exportacao a função de multiplicação

for index, row in df_exp.iterrows():
  # Atualiza a célula da linha atual (index) e coluna 'producao' usando loc
  df_exp.loc[index, ['exportacao_kg', 'exportacao_dol']] = mult(row['kilos'], row['dollar'])

In [ ]:
df_exp

,ano,kilos,dollar,pais_pt,cod_pais,exportacao_kg,exportacao_dol
0,2008-01-01,0,0,Afeganistão,AFG,0,0
1,2008-01-01,0,0,África do Sul,ZAF,0,0
2,2008-01-01,265742,429970,"Alemanha, República Democrática",DEU,58463,94593
3,2008-01-01,25721,71083,Angola,AGO,4630,12795
4,2008-01-01,0,0,Anguilla,AIA,0,0
...,...,...,...,...,...,...,...
2187,2023-01-01,0,0,Tuvalu,TUV,0,0
2188,2023-01-01,326093,454271,Uruguai,URY,65219,90854
2189,2023-01-01,0,0,Vanuatu,VUT,0,0
2190,2023-01-01,141030,220512,Venezuela,VEN,31027,48513


In [ ]:
#dropa colunas não utilizada

df_exp.drop(['kilos','dollar'], axis = 1, inplace = True)

# Estimando a Produção da Venti della Valle

In [ ]:
df_prod = df_exp.copy()

In [ ]:
# Mantém apenas colunas necessárias

df_prod = df_prod.groupby('ano')[['exportacao_kg', 'exportacao_dol']].sum()

# Tranforma novamente em dataframe

df_prod = df_prod.reset_index()

In [ ]:
df_prod.head()

,ano,exportacao_kg,exportacao_dol
0,2008-01-01,2103839,1496360
1,2009-01-01,5107165,1824278
2,2010-01-01,288096,572514
3,2011-01-01,257461,754813
4,2012-01-01,1123531,1111200


In [ ]:
''' Temos o quanto exportamos, aleatoriamente aplicamos um percentual entre 37% a 42% representando o que é exportado
 a diferença é a nossa produção'''

def prod(x,y):
  g = (rd.randint(37, 42)/100)
  return (g * x , g * y)

In [ ]:
#Itera sobre o index, de forma a aplicar para cada index na coluna criada chamada produção a função de prod

for index, row in df_prod.iterrows():
  # Atualiza a célula da linha atual (index) e coluna 'producao' usando loc
    df_prod.loc[index, ['producao_kg', 'producao_dol']] = mult(row['exportacao_kg'], row['exportacao_dol'])

In [ ]:
df_prod.head()

,ano,exportacao_kg,exportacao_dol,producao_kg,producao_dol
0,2008-01-01,2103839,1496360,399729,284308
1,2009-01-01,5107165,1824278,1021433,364856
2,2010-01-01,288096,572514,66262,131678
3,2011-01-01,257461,754813,46343,135866
4,2012-01-01,1123531,1111200,213471,211128


In [ ]:
#dropa colunas não utilizada

df_prod.drop(['exportacao_kg','exportacao_dol'], axis = 1, inplace = True)

In [ ]:
pd.DataFrame(df_prod)

,ano,producao_kg,producao_dol
0,2008-01-01,399729,284308
1,2009-01-01,1021433,364856
2,2010-01-01,66262,131678
3,2011-01-01,46343,135866
4,2012-01-01,213471,211128
5,2013-01-01,342606,857800
6,2014-01-01,93868,302005
7,2015-01-01,44578,105358
8,2016-01-01,95358,230186
9,2017-01-01,112518,295004


In [ ]:
# Proporções base para os diferentes tipos de vinho
base_proporcoes = np.array([0.4, 0.3, 0.2, 0.1])  # Vinho tinto, branco, rosé, espumantes

# Definindo uma função para gerar proporções com pequena variação
def gerar_proporcoes(base, variacao = (rd.uniform(1.5, 2.2)/100)):
    variacao_aleatoria = np.random.uniform(-variacao, variacao, size=base.shape)
    nova_proporcao = base + variacao_aleatoria
    nova_proporcao = nova_proporcao / nova_proporcao.sum()  # Normaliza para que a soma seja 1
    return nova_proporcao

In [ ]:
# Criando colunas para as categorias de vinho
for i, row in df_prod.iterrows():
    proporcoes = gerar_proporcoes(base_proporcoes)
    df_prod.loc[i, 'vinho_tinto_kg'] = row['producao_kg'] * proporcoes[0]
    df_prod.loc[i, 'vinho_branco_kg'] = row['producao_kg'] * proporcoes[1]
    df_prod.loc[i, 'vinho_rose_kg'] = row['producao_kg'] * proporcoes[2]
    df_prod.loc[i, 'espumantes_kg'] = row['producao_kg'] * proporcoes[3]
    df_prod.loc[i, 'vinho_tinto_dol'] = row['producao_dol'] * proporcoes[0]
    df_prod.loc[i, 'vinho_branco_dol'] = row['producao_dol'] * proporcoes[1]
    df_prod.loc[i, 'vinho_rose_dol'] = row['producao_dol'] * proporcoes[2]
    df_prod.loc[i, 'espumantes_dol'] = row['producao_dol'] * proporcoes[3]

In [ ]:
# Dropa colunas não utilziadas

df_prod.drop(['producao_kg','producao_dol'], axis = 1, inplace = True)

In [ ]:
df_prod.head()

,ano,vinho_tinto_kg,vinho_branco_kg,vinho_rose_kg,espumantes_kg,vinho_tinto_dol,vinho_branco_dol,vinho_rose_dol,espumantes_dol
0,2008-01-01,166200,117960,79635,35934,118210,83899,56641,25558
1,2009-01-01,413238,316721,199545,91929,147609,113133,71277,32837
2,2010-01-01,26146,19455,13149,7513,51957,38661,26130,14931
3,2011-01-01,18588,14016,9234,4506,54495,41090,27071,13210
4,2012-01-01,87471,63586,42416,19999,86511,62888,41950,19779


In [ ]:
# Reshape do dataframe

# 1º transpor kilos para linha

df_a = df_prod.melt(
    id_vars = ['ano'],
    value_vars = df_prod.filter(like = '_kg').columns,
    var_name = 'tipo_vinho',
    value_name = 'producao_kg'
    )

df_b = df_prod.melt(
    id_vars = ['ano'],
    value_vars = df_prod.filter(like = '_dol').columns,
    var_name = 'tipo_vinhos',
    value_name = 'producao_dol'
    )

# 3º sincronizar os índices

df_a = df_a.set_index(['ano', df_a.groupby(['ano']).cumcount()])
df_b = df_b.set_index(['ano', df_b.groupby(['ano']).cumcount()])

# 4º Juntar data_frames

df_c = (
    pd.concat([df_a, df_b], axis=1)
  .sort_index(level=1)
  .reset_index(level=1, drop=True)
  .reset_index()
  )

df_c.head()

# referências:
# https://stackoverflow.com/questions/51519101/simultaneously-melt-multiple-columns-in-python-pandas
# https://pandas.pydata.org/docs/reference/api/pandas.melt.html
# https://medium.com/horadecodar/data-science-tips-02-como-usar-loc-e-iloc-no-pandas-fab58e214d87

,ano,tipo_vinho,producao_kg,tipo_vinhos,producao_dol
0,2008-01-01,vinho_tinto_kg,166200,vinho_tinto_dol,118210
1,2009-01-01,vinho_tinto_kg,413238,vinho_tinto_dol,147609
2,2010-01-01,vinho_tinto_kg,26146,vinho_tinto_dol,51957
3,2011-01-01,vinho_tinto_kg,18588,vinho_tinto_dol,54495
4,2012-01-01,vinho_tinto_kg,87471,vinho_tinto_dol,86511


In [ ]:
#dropa colunas não utilizada

df_c.drop(['tipo_vinhos'], axis = 1, inplace = True)

# Replace no tipo_vinho

df_c['tipo_vinho'] = df_c['tipo_vinho'].str.replace('_kg', '')

In [ ]:
df_c.head()

,ano,tipo_vinho,producao_kg,producao_dol
0,2008-01-01,vinho_tinto,166200,118210
1,2009-01-01,vinho_tinto,413238,147609
2,2010-01-01,vinho_tinto,26146,51957
3,2011-01-01,vinho_tinto,18588,54495
4,2012-01-01,vinho_tinto,87471,86511


# Salva Arquivos Gerados

In [ ]:
##Salva Arquivo

#saída dados de exportacao
df_exp.to_excel('/content/drive/Othercomputers/Meu laptop/Fiap - Scripts/01 - Tech_Challenge/01 - Dados/exportacao_total_venti_della_valle.xlsx', index = False)

#saída dados de producao
df_c.to_excel('/content/drive/Othercomputers/Meu laptop/Fiap - Scripts/01 - Tech_Challenge/01 - Dados/producao_venti_della_valle.xlsx', index = False)